# Imports & Config

In [1]:
# Standard library imports
import os
import tempfile

# Third-party imports for data manipulation
import pandas as pd
import numpy as np

# Third-party imports for machine learning
import joblib
from sklearn.impute import SimpleImputer

# Third-party imports for data visualization
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# Third-party imports for video processing and computer vision
import cv2
import mediapipe as mp

# IPython imports for interactive widgets and display utilities
import ipywidgets as widgets
from IPython.display import Video, HTML, display

# Local imports for video processing pipeline
from src.pipeline.tracking.video_processing_pipeline import process_video, body_parts

# configuration
tracking_data_dir = './src/pipeline/tracking_data'
output_video_dir = './src/pipeline/output_videos'

# Ensure all directories exist
dirs_to_check = [tracking_data_dir, output_video_dir]
for dir in dirs_to_check:
    if not os.path.exists(dir):
        os.makedirs(dir)
        print(f"Directory created: {dir}")

/Users/linus/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Video Tracking

In [4]:
# Define the file upload widget
file_selector = widgets.FileUpload(
    accept='.mp4, .mov',  # Specify file types
    multiple=False,  # Allow multiple files to be selected
    description='Select a video file',
    layout={'width': '400px'}
)

# Define a button widget
button = widgets.Button(
    description='Process Uploaded File',
    button_style='',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to process the uploaded file',
    icon='play',  # Button icon
    layout={'width': '400px'}
)

def save_uploaded_file(uploaded_file):
    file_name = uploaded_file['name']
    file_content = uploaded_file['content']
    with tempfile.NamedTemporaryFile(delete=False, suffix=file_name) as tmp_file:
        tmp_file.write(file_content)
        return tmp_file.name, os.path.dirname(tmp_file.name)

def save_landmarks_on_video(video_path, output_video_path, tracking_data_path):
    print("Creating video with landmarks... (this might take a while)")

    # Initialize MediaPipe Pose
    mp_pose = mp.solutions.pose
    mp_drawing = mp.solutions.drawing_utils

    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    # Check if video opened successfully
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Get the width, height, and frame rate of the video frame
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Read the tracking data from CSV
    tracking_data = pd.read_csv(tracking_data_path)

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'avc1')  # or 'XVID'
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    # Process video and draw landmarks from tracking data
    for index, row in tracking_data.iterrows():
        # Read frame from video
        ret, frame = cap.read()
        if not ret:
            break

        # Draw landmarks on the frame
        for landmark in mp.solutions.pose.PoseLandmark:
            landmark_name = landmark.name.lower().replace('_', ' ')  # Convert to lowercase and replace underscores with spaces
            landmark_name = landmark_name.capitalize()  # Capitalize only the first letter of the first word
            landmark_x = f'{landmark_name}_x'
            landmark_y = f'{landmark_name}_y'
            if landmark_x in tracking_data.columns and landmark_y in tracking_data.columns and not pd.isna(row[landmark_x]) and not pd.isna(row[landmark_y]):
                x = int(row[landmark_x] * width)
                y = int(row[landmark_y] * height)
                # Check if the current landmark is one of the hips
                if landmark in [mp.solutions.pose.PoseLandmark.LEFT_HIP, mp.solutions.pose.PoseLandmark.RIGHT_HIP]:
                    # Highlight hips in red
                    cv2.circle(frame, (x, y), 15, (0, 0, 255), -1)
                else:
                    # Draw other landmarks in green
                    cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)


        # Write the frame with landmarks to the output video file
        out.write(frame)

    # Release everything when job is finished
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print(f"Video saved to {output_video_path}")

def process_and_display_tracking_data(tmp_file_path, tmp_file_dir):
    print("Processing video... (this might take a while)")
    process_video(os.path.basename(tmp_file_path), tmp_file_dir, tracking_data_dir)
    tracking_data_file = f'Tracking_{os.path.basename(tmp_file_path[:-4])}.csv'
    tracking_data_df = pd.read_csv(os.path.join(tracking_data_dir, tracking_data_file))
    display(tracking_data_df.head())
    # After tracking data is processed, save video with landmarks
    output_video_path = os.path.join(output_video_dir, f"{os.path.basename(tmp_file_path[:-4])}_landmarks.mp4")
    save_landmarks_on_video(tmp_file_path, output_video_path, os.path.join(tracking_data_dir, tracking_data_file))
    video_html = f'''
    Video with landmarks saved to 
    <a href='{output_video_path}' target='_blank'>{output_video_path}</a> <br><br>
    <video height="500" controls allowfullscreen>
      <source src="{output_video_path}" type="video/mp4">
    Your browser does not support the video tag.
    </video>
    '''
    display(HTML(video_html))

# Define an event handler for the button click event
def on_button_clicked(b):
    uploaded_files = file_selector.value
    if uploaded_files:
        uploaded_file = next(iter(uploaded_files))
        tmp_file_path, tmp_file_dir = save_uploaded_file(uploaded_file)
        process_and_display_tracking_data(tmp_file_path, tmp_file_dir)
    else:
        print("No file uploaded.")

# Attach the event handler to the button
button.on_click(on_button_clicked)

# Display the widgets
display(file_selector, button)


FileUpload(value=(), accept='.mp4, .mov', description='Select a video file', layout=Layout(width='400px'))

Button(description='Process Uploaded File', icon='play', layout=Layout(width='400px'), style=ButtonStyle(), to…

INFO:root:Processing video: tmpbbw2er76IMG_9340.MOV


Processing video... (this might take a while)


INFO:root:Tracking data for tmpbbw2er76IMG_9340.MOV stored in ./src/pipeline/tracking_data/Tracking_tmpbbw2er76IMG_9340.csv


,Label,Timestamp,Nose_x,Nose_y,Nose_z,Left eye inner_x,Left eye inner_y,Left eye inner_z,Left eye_x,Left eye_y,...,Left heel_z,Right heel_x,Right heel_y,Right heel_z,Left foot index_x,Left foot index_y,Left foot index_z,Right foot index_x,Right foot index_y,Right foot index_z
0,NaN,0,0.447181,0.186625,-0.454502,0.466466,0.167357,-0.481824,0.478228,0.166733,...,0.208295,0.527764,0.884256,0.747109,0.546632,0.973369,-0.081372,0.389055,0.922492,0.568898
1,NaN,1,0.447440,0.189193,-0.463861,0.467123,0.170478,-0.498253,0.478337,0.169845,...,0.250730,0.527312,0.884385,0.692368,0.548656,0.978809,-0.046427,0.390017,0.923248,0.520040
2,NaN,2,0.447379,0.191168,-0.478758,0.467114,0.172948,-0.515375,0.478157,0.172240,...,0.243222,0.527092,0.884469,0.687063,0.551741,0.980843,-0.052328,0.390481,0.923249,0.517358
3,NaN,3,0.447204,0.191382,-0.487665,0.467088,0.173184,-0.523442,0.478086,0.172532,...,0.245198,0.527091,0.885141,0.690187,0.553133,0.981776,-0.046310,0.390478,0.923281,0.523120
4,NaN,4,0.446104,0.191954,-0.475087,0.465854,0.173861,-0.520072,0.476621,0.173205,...,0.256576,0.527125,0.885262,0.690394,0.554150,0.982438,-0.032706,0.390477,0.923279,0.525715


Creating video with landmarks... (this might take a while)
Video saved to ./src/pipeline/output_videos/tmpbbw2er76IMG_9340_landmarks.mp4


# Model Usage - Squat Phase Analysis

# Model Training